Random Search: The Grid Search one that we have discussed above usually increases the complexity in terms of the computation flow, So sometimes GS is considered inefficient since it attempts all the combinations of given hyperparameters.  But the Randomized Search is used to train the models based on random hyperparameters and combinations. obviously, the number of training models are small column than grid search.

In simple terms, In Random Search, in a given grid, the list of hyperparameters are trained and test our model on a random combination of given hyperparameters.



For y variable encoding is done as Poisonous = p -> 1 Edible = e -> 0



Abbaspour - 610398147 - HW1

We're going to see how decision tree classifying works for mushroom dataset.
At first we import necessary methods from useful libraries.
Time for computing how long does modeling takes.
Some methods from 'pandas' to acess .csv format dataframe.
some sklearn methods for manipulating data located in dataset and evaluate what would be done.
Matplotlib and seaborn methods are used for plotting.

In [ ]:
from time import time
from pandas import read_csv, get_dummies # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_text #, plot_tree, export_graphviz
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from matplotlib.pyplot import subplots, title, fill_between, xlabel, ylabel, figure, show
from seaborn import histplot, heatmap, countplot

This hand-made funciton is called four times for four states 'GridSearchCV' and 'RandomizedSearchCV' for two sort of data preprocessings: 'One Hot Encoding' (which causes cursed dimensionality from 22 to 117) and 'Label Encoding Only'.
The function has the raw Model for hyper parameterizaiton(Fine tuning) and first, optimize the parameters by being applied with validation set and then, gets trained with train set.
At last, evaluation parameters and also confusion matrices for each splitted dataset will be contructed.
Also texual representation for each decision tree is available.
Despite figuring out best possible criteria and maximum depths, it finds out optimal values for other parameters as bonus.

In [ ]:
def Modeling(Model):
    startTime = time()
    # Obtaining best paramteres by using splitted validation data
    Model.fit(X_val, y_val)
    Duration = time() - startTime # Calculating how long fitting train data takes

    print(f"\n{Model.best_score_}")
    print(f"\n{Model.best_params_}")
    Model = Model.best_estimator_ # = DecisionTreeClassifier(Optimal Parameters)
    print(f"\n{Model}")

    Model.fit(X_train, y_train)
    y_pred = Model.predict(X_test)
    print(f"\nModel Score: {Model.score(X_test, y_pred)}")
    print(f"\nAccuracy Score: {accuracy_score(y_test, y_pred)}" )
    print(f"\nPrecision Score: {precision_score(y_test, y_pred)}")
    print(f"\nRecall Score: {recall_score(y_test, y_pred)}")
    print(f"\nF1-Score: {f1_score(y_test,y_pred)}")
    print(f"\nComputation Time: {Duration}")

    # Confusion matrix for train set
    _pred = Model.predict(X_train)
    ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_train, _pred), display_labels = ["True", "False"]).plot()
    show()

    # Confusion matrix for validation set
    _pred = Model.predict(X_val)
    ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_val, _pred), display_labels = ["True", "False"]).plot()
    show()

    # Confusion matrix for test set
    ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, y_pred), display_labels = ["True", "False"]).plot()
    show()

    print("Decision Tree Classifier report: \n\n", classification_report(y_test, y_pred))

    # Texual representation of modelled decision tree
    print(export_text(Model))

Opening main dataset and plotting basic informations about it containing class distribution graph.

In [ ]:
df = read_csv("mushrooms.csv").copy()

# Decribing raw data records
nSamples, nFeatures = df.shape
print(f"Data record has {nSamples} samples and {nFeatures} features.\n")
df.describe()
df.info()
df.isnull().values.any() # Check if any missing value exists.

# Plotting raw data records
Figure, ax = subplots()
ax.pie(df["class"].value_counts(), labels = ["Edible", "Poisonous"], autopct = '%1.1f%%', shadow = True, startangle = 90)
ax.axis('equal')
show()
histplot(df['class'])
show()

Now we model two fine tuning methods for two preprocessed splitted data.
Dataset is splitted by 70, 15, 15

In [ ]:
# Preprocessing data by means of Label Enconding & One Hot Encoding
labelEncoder, Features = LabelEncoder(), df.columns
X_train, X_test, y_train, y_test = train_test_split(get_dummies(df.drop(['class'], axis = 1)), labelEncoder.fit_transform(df['class']), test_size = 0.3, random_state = 42) # 70:30

# second, split into X_val, X_test, y_val, y_test
# 'test_size=0.5' split into 50% and 50%. The original data set is 30%; so, it will split into 15% equally.
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42) # 70:15:15

# splitted dataset dimensions
print("\nDimension of splitted datasets are listed below:")
print(f"\nTrain sets: {X_train.shape}", y_train.shape)
print(f"\nValidation sets: {X_val.shape}", y_val.shape)
print(f"\nTest sets: {X_test.shape}", y_test.shape)

# Train model by Grid Search hyper parameterization approach
print("\nGridSearchCV:")
Modeling(GridSearchCV(estimator = DecisionTreeClassifier(random_state = 42), param_grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}, verbose = 1, cv = 3, scoring = "accuracy", error_score= "raise"))

# Train model by Randomized Search hyper parameterization approach
print("\nRandomizedSearchCV:")
Modeling(RandomizedSearchCV(DecisionTreeClassifier(), {"criterion": ["gini", "entropy"], "max_features": ["sqrt", "log2"], "min_samples_leaf": range(1, 100, 1), "max_depth": range(1, 50, 1)}, cv = 10, scoring = "accuracy", n_iter = 20, random_state = 5))

# Label Enconding Only
for Feature in Features:
    df[Feature] = labelEncoder.fit_transform(df[Feature])

# Convert data to have zero mean and unit variance
X_train, X_test, y_train, y_test = train_test_split(StandardScaler().fit_transform(df.iloc[:,1:18]), df.iloc[:, 0], test_size=0.3, random_state = 42) # 70:30

# 'test_size=0.5' split into 50% and 50%. The original data set is 30%; so, it will split into 15% equally.
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42) # 70:15:15

# Train model by Grid Search hyper parameterization approach
print("\nGridSearchCV:")
Modeling(GridSearchCV(estimator = DecisionTreeClassifier(random_state = 42), param_grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}, verbose = 1, cv = 3, scoring = "accuracy", error_score= "raise"))

# Train model by Randomized Search hyper parameterization approach
print("\nRandomizedSearchCV:")
Modeling(RandomizedSearchCV(DecisionTreeClassifier(), {"criterion": ["gini", "entropy"], "max_features": ["sqrt", "log2"], "min_samples_leaf": range(1, 100, 1), "max_depth": range(1, 50, 1)}, cv = 10, scoring = "accuracy", n_iter = 20, random_state = 5))


Data Analysis
In the prior data set, edible mushrooms were recorded a bit more than poisonous mushrooms were.
Any detail about whole results obtained from generating whole modeling types are illustrated in 'Evalutaions' folder.
According to that:
Most accurate confusion matrix for:
1. Train set was in RandomizedSearchCV for full-preprocessed data and GridSearchCV for label-encoded only data modeling types
2. Validation set was in GridSearchCV for label-encoded only data modeling type
3. Test set was in RandomizedSearchCV for full-preprocessed data modeling type

Least accurate confusion matrix for:
1. Train set was in GridSearchCV for full-preprocessed data modeling type
2. Validation set was in RandomizedSearchCV for full-preprocessed data modeling type
3. Test set was in GridSearchCV for full-preprocessed data modeling type

According to their scores, the best modeling type is GridSearchCV for full-preprocessed data modeling and the worst one is RandomizedSearchCV for label-encoded only data.

When working with a mushroom dataset, especially for classification tasks such as determining whether a mushroom is edible or poisonous, selecting the most informative features for comparison is crucial. Here are some of the best features typically used in the analysis of mushroom datasets:

1. **Cap Shape:** The shape of the mushroom cap (e.g., bell, conical, flat, convex) can be a significant feature for classification.

2. **Cap Surface:** The surface texture of the mushroom cap (e.g., fibrous, grooves, scaly, smooth) can provide important information.

3. **Cap Color:** The color of the mushroom cap is a key feature for identification and classification.

4. **Bruises:** Whether the mushroom bruises when damaged can be a distinguishing characteristic.

5. **Odor:** Mushroom odor (e.g., almond, anise, creosote, fishy, foul) is a critical feature for differentiating between species.

6. **Gill Attachment:** The attachment of gills to the stem (e.g., free, attached, notched) is a relevant feature.

7. **Gill Spacing:** The spacing between gills on the underside of the cap (e.g., close, crowded, distant) can be informative.

8. **Gill Size:** The size of gills in relation to the cap diameter is a valuable feature.

9. **Stalk Shape:** The shape of the mushroom stalk (e.g., enlarging, tapering) is an important characteristic.

10. **Stalk Surface Above Ring:** The texture of the stalk surface above the ring (e.g., fibrous, scaly, smooth) provides useful insights.

11. **Stalk Color Above Ring:** The color of the stalk above the ring is a distinguishing feature.

12. **Stalk Color Below Ring:** Similarly, the color of the stalk below the ring can be indicative of species.

13. **Veil Color:** The color of the veil covering the gills (e.g., brown, orange, white) is a relevant feature.

14. **Ring Number:** The number of rings on the stalk can be a distinguishing feature.

15. **Spore Print Color:** The color of the spore print left by the mushroom can aid in identification.

When comparing features in a mushroom dataset, it is essential to select those that are most discriminatory and relevant for distinguishing between different classes (edible vs. poisonous). Feature selection techniques such as correlation analysis, mutual information, and feature importance from machine learning models can help identify the most informative features in the dataset. Conducting exploratory data analysis and considering domain knowledge can also provide insights into the best features for comparison in a mushroom dataset.

Challenges:
Main challenge was to preprocess raw data and made features numerical.
Minor challenge was about plotting.
